<font size="5"> 03 Introduction to Logistic Regression STA 6238</font> <br>\
<font size="5"> Mina Akhondzadeh</font>

The dataset is about loan eligibility prediction. I downloaded it from Kaggle: <br>
https://www.kaggle.com/datasets/vikasukani/loan-eligible-dataset?resource=download

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


**Read in data**

In [2]:
df = read_csv("loan_dataset.csv")
df$Loan_Status = as.factor(df$Loan_Status)
head(df)

Rows: 480 Columns: 11
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (6): Gender, Married, Dependents, Education, Self_Employed, Property_Area
dbl (5): ApplicantIncome, LoanAmount, Loan_Amount_Term, Credit_History, Loan...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
Male,Yes,1,Graduate,No,4583,128,360,1,Rural,0
Male,Yes,0,Graduate,Yes,3000,66,360,1,Urban,1
Male,Yes,0,Not Graduate,No,2583,120,360,1,Urban,1
Male,No,0,Graduate,No,6000,141,360,1,Urban,1
Male,Yes,2,Graduate,Yes,5417,267,360,1,Urban,1
Male,Yes,0,Not Graduate,No,2333,95,360,1,Urban,1


**Logistic Regression Model on selected features**

In [3]:
glm_loan = glm(
        formula = Loan_Status ~ Credit_History + Property_Area + Education, 
        data = df,
        family = binomial()
)

In [4]:
summary(glm_loan)


Call:
glm(formula = Loan_Status ~ Credit_History + Property_Area + 
    Education, family = binomial(), data = df)

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.6246     0.4552  -5.766  8.1e-09 ***
Credit_History           3.6037     0.4247   8.485  < 2e-16 ***
Property_AreaSemiurban   0.9692     0.2948   3.287  0.00101 ** 
Property_AreaUrban       0.2290     0.2851   0.803  0.42197    
EducationNot Graduate   -0.2424     0.2884  -0.840  0.40075    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 593.05  on 479  degrees of freedom
Residual deviance: 450.69  on 475  degrees of freedom
AIC: 460.69

Number of Fisher Scoring iterations: 4


In [5]:
#Create columns for predicted probability and predicted class
df <- df %>% mutate(
            predicted_loan = predict(
                glm_loan,
                type = "response"
            ),
            predicted_class = if_else(
                condition = predicted_loan > 0.5,
                true = 1,
                false = 0
            ),
            predicted_class = as.factor(predicted_class)
        )

**Confusion Matrix**

In [6]:
caret::confusionMatrix(
            data = df$predicted_class, 
            reference = df$Loan_Status
        )

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0  63   7
         1  85 325
                                          
               Accuracy : 0.8083          
                 95% CI : (0.7702, 0.8426)
    No Information Rate : 0.6917          
    P-Value [Acc > NIR] : 5.209e-09       
                                          
                  Kappa : 0.4738          
                                          
 Mcnemar's Test P-Value : 9.923e-16       
                                          
            Sensitivity : 0.4257          
            Specificity : 0.9789          
         Pos Pred Value : 0.9000          
         Neg Pred Value : 0.7927          
             Prevalence : 0.3083          
         Detection Rate : 0.1313          
   Detection Prevalence : 0.1458          
      Balanced Accuracy : 0.7023          
                                          
       'Positive' Class : 0               
                              

**Model Diagnostics: checking multicolinearity**

In [7]:
car::vif(
        mod = glm_loan
    )

,GVIF,Df,GVIF^(1/(2*Df))
Credit_History,1.019024,1,1.009467
Property_Area,1.026862,2,1.006649
Education,1.007761,1,1.003873
